# Model Testing with Faster GPU 

# Loading Dependencies

In [1]:
# Dependencies to Visualize the model
import tensorflow as tf
# Set memory growth
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

# Enable mixed precision
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

%matplotlib inline
from IPython.display import Image, SVG
import matplotlib.pyplot as plt
import numpy as np
# Setting seed for reproducibility
np.random.seed(0)

# Filepaths, pandas, numpy, Tensorflow, and scikit-image
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import skimage as sk

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti, compute capability 8.6


# Preprocessing of Images

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your image directory
image_directory = "Resources/Skin Cancer"
# Augmented Image Directory 
# image_directory = "resources_augmented/Skin Cancer"

# Defining original image size
image_size = (600, 450)

# Define the batch size
batch_size = 32

# EDIT THIS OUT 
# Create an ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    # rotation_range=20,  # Randomly rotate images by 20 degrees
    # width_shift_range=0.2,  # Randomly shift images horizontally by 20%
    # height_shift_range=0.2,  # Randomly shift images vertically by 20%
    # horizontal_flip=True,  # Randomly flip images horizontally
    # zoom_range=0.2,  # Randomly zoom images by 20%
)

# Load and preprocess the train dataset with data augmentation
train_dataset = train_datagen.flow_from_directory(
    directory=os.path.join(image_directory, "train"),  # Use the 'train' directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42,
)

# Create an ImageDataGenerator for validation data (no augmentation)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values

# Load and preprocess the validation dataset without data augmentation
val_dataset = val_datagen.flow_from_directory(
    directory=os.path.join(image_directory, "val"),  # Use the 'val' directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42,
)

def get_class_counts(dataset):
    class_labels = list(dataset.class_indices.keys())
    class_counts = dict(zip(class_labels, [0] * len(class_labels)))
    for _, labels in dataset:
        for label in labels:
            class_counts[class_labels[int(label.argmax())]] += 1
        if sum(class_counts.values()) >= dataset.samples:
            break
    return class_counts

train_counts = get_class_counts(train_dataset)
val_counts = get_class_counts(val_dataset)

# Combine the counts from training and validation
combined_counts = {class_name: train_counts.get(class_name, 0) + val_counts.get(class_name, 0) for class_name in set(train_counts) | set(val_counts)}
total_samples = sum(combined_counts.values())

print("\nOverall Dataset Class Distribution:")
print(f"Total samples across train and validation datasets: {total_samples}")
for class_name, count in combined_counts.items():
    percentage = (count / total_samples) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

# Print class distribution for training and validation datasets individually
print("\nTrain Dataset Class Distribution:")
for class_name, count in train_counts.items():
    percentage = (count / train_dataset.samples) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

print("\nValidation Dataset Class Distribution:")
for class_name, count in val_counts.items():
    percentage = (count / val_dataset.samples) * 100
    print(f"{class_name}: {count} ({percentage:.2f}%)")

# Print the class names and dataset shapes
print("\nSummary:")
print("Class Names:", list(train_dataset.class_indices.keys()))
print("Train Dataset Shape:", train_dataset.image_shape)
print("Validation Dataset Shape:", val_dataset.image_shape)

Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.

Overall Dataset Class Distribution:
Total samples across train and validation datasets: 10015
df: 115 (1.15%)
bkl: 1099 (10.97%)
mel: 1113 (11.11%)
akiec: 327 (3.27%)
nv: 6705 (66.95%)
bcc: 514 (5.13%)
vasc: 142 (1.42%)

Train Dataset Class Distribution:
akiec: 262 (3.27%)
bcc: 411 (5.13%)
bkl: 879 (10.97%)
df: 92 (1.15%)
mel: 890 (11.11%)
nv: 5364 (66.95%)
vasc: 114 (1.42%)

Validation Dataset Class Distribution:
akiec: 65 (3.25%)
bcc: 103 (5.14%)
bkl: 220 (10.98%)
df: 23 (1.15%)
mel: 223 (11.13%)
nv: 1341 (66.95%)
vasc: 28 (1.40%)

Summary:
Class Names: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Train Dataset Shape: (600, 450, 3)
Validation Dataset Shape: (600, 450, 3)


# Continuous Testing 
This section contains a conglomerated model run. It was created so that the models could be tested overnight. I'm going to bed now.

In [3]:
# Set GPU options
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)  # Adjust the fraction as needed
session = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [4]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Activation, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2

import csv
import os
import numpy as np

from sklearn.utils import class_weight

# ///////////////////////////////////////////////////////////////////////////////////////////
# CHECK RUN NUMBER BEFORE RUNNING
# Run Number - use to create new directory or add to existing directory
run_number = 17
run_dir = f"run{run_number}"
os.makedirs(run_dir, exist_ok=True)

# ///////////////////////////////////////////////////////////////////////////////////////////


# Creating a CSV file to store the model results
csv_file = "model_results.csv"
fieldnames = [
    "architecture",
    "optimizer",
    "loss",
    "accuracy",
    "precision",
    "recall",
    "auc",
]

# Write the header to the CSV file. Data is written to the file after model_result
with open(csv_file, mode="w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

batch_size = 64
architectures = ["InceptionV3"]  # Options: "VGG16", "ResNet50", "InceptionV3", "PNASNet5Large", "InceptionResNetV2"
optimizers = ["Adam"]  # Options: "Adam", "RMSprop", "SGD"

# Create directory to store models
models_dir = f"{run_dir}/models"
os.makedirs(models_dir, exist_ok=True)

# Initialize model_results to store the evaluation results
model_results = []

for architecture in architectures:
    # Set the input shape and preprocessing function based on the selected architecture
    if architecture == "VGG16" :
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    elif architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.resnet50.preprocess_input
    elif architecture == "InceptionV3":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input
    elif architecture == "InceptionResNetV2":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_resnet_v2.preprocess_input

    # Load and preprocess data using tf.keras.preprocessing
    # This adds data augmentation to the training dataset
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        # rotation_range=30,
        # width_shift_range=0.3,
        # height_shift_range=0.3,
        # horizontal_flip=True,
        # zoom_range=0.3,
    )

    train_dataset = train_datagen.flow_from_directory(
        "Resources/Skin Cancer/train",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

    # Define the validation dataset without data augmentation
    val_dataset = val_datagen.flow_from_directory(
        "Resources/Skin Cancer/val",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    # Define the pre-trained model architecture. Will select proper model based on current 'architecture' in for loop
    if architecture == "VGG16":
        base_model = VGG16(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "ResNet50":
        base_model = ResNet50(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "InceptionV3":
        base_model = InceptionV3(
            weights="imagenet", include_top=False, input_shape=input_shape
        )
    elif architecture == "InceptionResNetV2":
        base_model = InceptionResNetV2(
            weights="imagenet", include_top=False, input_shape=input_shape, pooling='avg'
            )
        
    # Freeze the layers of the pre-trained model
    for layer in base_model.layers:
        layer.trainable = False

    # Get the number of unique classes from the train_dataset
    num_classes = len(train_dataset.class_indices)

   # Flag to control whether to use the custom layers or not
    use_custom_layers = True

    if use_custom_layers:
        # Add regular custom layers on top of the pre-trained model
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, activation="relu")(x)
        x = Dropout(0.5)(x)
        predictions = Dense(num_classes, activation="softmax")(x)
    else:
        # Use global average pooling and a dense layer for predictions
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        predictions = Dense(num_classes, activation="softmax")(x)

    
    # Create the final model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Step through and adjust for each optimizer
    for optimizer_name in optimizers:
        # Define the optimizer
        if optimizer_name == "Adam":
            optimizer = Adam(learning_rate=0.01)
        elif optimizer_name == "RMSprop":
            optimizer = RMSprop(learning_rate=0.01)
        elif optimizer_name == "SGD":
            optimizer = SGD(learning_rate=0.01, momentum=0.9)

        # Create learning rate scheduler that monitors validation loss
        lr_scheduler = ReduceLROnPlateau(
            monitor="val_loss", factor=0.1, patience=5, verbose=1
        )

       
       # Compile the model
        model.compile(
            optimizer=optimizer,
            loss="categorical_crossentropy",
            metrics=["accuracy", Precision(), Recall(), AUC()],
        )
        # Create a TensorBoard callback with a separate log directory for each model and optimizer
        tensorboard_callback = TensorBoard(
            log_dir=f"./{run_dir}/logs/{architecture}_{optimizer_name}",
            histogram_freq=1,
        )

        # Create an EarlyStopping callback to prevent overfitting
        early_stopping = EarlyStopping(
            monitor="val_loss", patience=5, restore_best_weights=True
        )

        # Create a ModelCheckpoint callback to save the best model weights
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}.weights.h5"
            ),
            save_weights_only=True,
            save_best_only=True,
            monitor="val_loss",
            verbose=1,
        )

        # Create a CSVLogger callback with keras
        csv_logger = CSVLogger(
            os.path.join(
                models_dir, f"model_{architecture}_{optimizer_name}_training.csv"
            )
        )

        # Calculate class weights using sklearn's class_weight
        class_weights = class_weight.compute_class_weight(
            'balanced',
            classes=np.unique(train_dataset.classes),
            y=train_dataset.classes
        )
        class_weight_dict = dict(enumerate(class_weights))
      
        # Train the model with the checkpoint callback and weights
        epochs = 25
        history = model.fit(
            train_dataset,
            steps_per_epoch=train_dataset.samples // batch_size,
            validation_data=val_dataset,
            validation_steps=val_dataset.samples // batch_size,
            epochs=epochs,
            class_weight=class_weight_dict,
            callbacks=[
                tensorboard_callback,
                lr_scheduler,
                checkpoint_callback,
                csv_logger,
            ],
        )

        # Save the optimizer state to avoid retraining the model
        model.save(
            os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5")
        )

         # Evaluate the model on the validation set and print the results
        loss, accuracy, precision, recall, auc = model.evaluate(val_dataset)
        print(f"Model: {architecture}, Optimizer: {optimizer_name}")
        print(f"Validation Loss: {loss:.4f}")
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(f"Validation Precision: {precision:.4f}")
        print(f"Validation Recall: {recall:.4f}")
        print(f"Validation AUC-ROC: {auc:.4f}")
        print(class_weights)
        print(class_weight_dict)

        # Append the model results to the list
        model_result = {
            "architecture": architecture,
            "optimizer": optimizer_name,
            "loss": loss,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "auc": auc,
        }
        model_results.append(model_result)

        # Write the current model result to the CSV file
        with open(csv_file, mode="a", newline="") as file:
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writerow(model_result)

        # Save the model
        model.save(
            os.path.join(models_dir, f"model_{architecture}_{optimizer_name}.h5")
        )

Found 8012 images belonging to 7 classes.
Found 2003 images belonging to 7 classes.
Epoch 1/25
125/125 [==============================] - ETA: 0s - loss: 5.7891 - accuracy: 0.2224 - precision: 0.2374 - recall: 0.0734 - auc: 0.6114
Epoch 1: val_loss improved from inf to 2.00992, saving model to run17/models\model_InceptionV3_Adam.weights.h5
125/125 [==============================] - 47s 337ms/step - loss: 5.7891 - accuracy: 0.2224 - precision: 0.2374 - recall: 0.0734 - auc: 0.6114 - val_loss: 2.0099 - val_accuracy: 0.0837 - val_precision: 0.2281 - val_recall: 0.0066 - val_auc: 0.5048 - lr: 0.0100
Epoch 2/25
125/125 [==============================] - ETA: 0s - loss: 1.7600 - accuracy: 0.1656 - precision: 0.2834 - recall: 0.0238 - auc: 0.6607
Epoch 2: val_loss improved from 2.00992 to 1.98239, saving model to run17/models\model_InceptionV3_Adam.weights.h5
125/125 [==============================] - 42s 333ms/step - loss: 1.7600 - accuracy: 0.1656 - precision: 0.2834 - recall: 0.0238 - auc:

# Evaluating and Visualizing the Data

In [6]:
# Importing Dependencies
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import TensorBoard, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import csv
import os

from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import os

model_results = "model_results.csv"

# Run Number - use to create new directory or add to existing directory
run_number = 17
run_dir = f"run{run_number}"
os.makedirs(run_dir, exist_ok=True)


architectures = ["InceptionV3"] # Options: "VGG16", "ResNet50", "InceptionV3", "InceptionResNetV2"
optimizers = ["Adam"] # Options: "Adam", "RMSprop", "SGD"

batch_size = 32

# Define the preprocessing function based on the architectures you want to visualize
architectures_to_visualize = ["InceptionV3"]  # Specify the architectures you want to visualize
preprocessing_functions = {
    "VGG16": tf.keras.applications.vgg16.preprocess_input,
    "ResNet50": tf.keras.applications.resnet50.preprocess_input,
    "InceptionV3": tf.keras.applications.inception_v3.preprocess_input,
    "InceptionResNetV2": tf.keras.applications.inception_resnet_v2.preprocess_input
}


# Create a directory to store the visualization results
visualizations_dir = f"{run_dir}/visualizations"
os.makedirs(visualizations_dir, exist_ok=True)

#

# Iterate over each model architecture and optimizer
for architecture in architectures:
    # Set the preprocessing function based on the architecture
    preprocessing_function = preprocessing_functions[architecture]

    # Set the input shape and preprocessing function based on the selected architecture
    if architecture == "VGG16":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.vgg16.preprocess_input
    elif architecture == "ResNet50":
        input_shape = (224, 224, 3)
        preprocessing_function = tf.keras.applications.resnet50.preprocess_input
    elif architecture == "InceptionV3":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_v3.preprocess_input
    elif architecture == "InceptionResNetV2":
        input_shape = (299, 299, 3)
        preprocessing_function = tf.keras.applications.inception_resnet_v2.preprocess_input

    # Redefine the validation dataset with the corresponding preprocessing function
    val_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)
    val_dataset = val_datagen.flow_from_directory(
        "Resources/Skin Cancer/val",
        target_size=input_shape[:2],
        batch_size=batch_size,
        class_mode="categorical",
    )

    # Get the class names from the redefined val_dataset
    class_names = list(val_dataset.class_indices.keys())


    for optimizer_name in optimizers:
        # Load the trained model
        model = load_model(os.path.join(f"{run_dir}/models/", f"model_{architecture}_{optimizer_name}.h5"))

        # Make predictions on the validation dataset using the redefined val_dataset
        y_pred = model.predict(val_dataset)
        y_pred_classes = np.argmax(y_pred, axis=1)

        # Get the true labels of the validation dataset
        y_true = val_dataset.classes

        # Compute the confusion matrix
        cm = confusion_matrix(y_true, y_pred_classes)
        print(f"Confusion Matrix for {architecture}_{optimizer_name}:")
        print(cm)

        # Save the confusion matrix as a CSV file
        cm_filename = f"confusion_matrix_{architecture}_{optimizer_name}.csv"
        np.savetxt(os.path.join(visualizations_dir, cm_filename), cm, delimiter=",")

        # Compute the classification report
        cr = classification_report(y_true, y_pred_classes, target_names=class_names)
        print(f"Classification Report for {architecture}_{optimizer_name}:")
        print(cr)

        # Save the classification report as a text file
        cr_filename = f"classification_report_{architecture}_{optimizer_name}.txt"
        with open(os.path.join(visualizations_dir, cr_filename), "w") as file:
            file.write(cr)

        # Compute the ROC curve and AUC for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(len(class_names)):
            fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Plot the ROC curve for each class
        plt.figure(figsize=(8, 6))
        for i in range(len(class_names)):
            plt.plot(
                fpr[i],
                tpr[i],
                label=f"ROC curve of class {class_names[i]} (AUC = {roc_auc[i]:.2f})",
            )
        plt.plot([0, 1], [0, 1], "k--")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve for {architecture}_{optimizer_name}")
        plt.legend(loc="lower right")
        plt.tight_layout()
        plt.savefig(os.path.join(visualizations_dir, f"roc_curve_{architecture}_{optimizer_name}.png"))
        plt.close()

        # Visualize the model's predictions on a subset of the validation data
        subset_size = 10
        subset_indices = np.random.choice(len(val_dataset), subset_size, replace=False)
        subset_images = []
        subset_labels = []
        val_dataset.reset()  # Reset the validation dataset iterator
        for i in range(len(val_dataset)):
            if i in subset_indices:
                image_batch, label_batch = next(val_dataset)
                for image, label in zip(image_batch, label_batch):
                    subset_images.append(image)
                    subset_labels.append(label)

        subset_images = np.array(subset_images)
        subset_labels = np.array(subset_labels)

        subset_preds = model.predict(subset_images)
        subset_pred_classes = np.argmax(subset_preds, axis=1)

        # Generate the visualization plot
        plt.figure(figsize=(15, 10))
        for i in range(subset_size):
            plt.subplot(2, 5, i + 1)
            plt.imshow(subset_images[i])
            plt.title(
                f"True: {class_names[np.argmax(subset_labels[i])]}\\nPred: {class_names[subset_pred_classes[i]]}"
            )
            plt.axis("off")
        plt.tight_layout()
        plt.savefig(os.path.join(visualizations_dir, f"predictions_{architecture}_{optimizer_name}.png"))
        plt.close()


# Read the CSV file and store the model results
model_results = []
with open("model_results.csv", "r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        model_results.append(row)

# Create a bar plot for validation precision
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["precision"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Precision")
plt.title("Validation Precision for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_precision_comparison.png"))
plt.close()

# Create a bar plot for validation recall
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["recall"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation Recall")
plt.title("Validation Recall for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_recall_comparison.png"))
plt.close()

# Create a bar plot for validation AUC-ROC
plt.figure(figsize=(10, 6))
plt.bar(range(len(model_results)), [float(result["auc"]) for result in model_results])
plt.xticks(
    range(len(model_results)),
    [f"{result['architecture']}_{result['optimizer']}" for result in model_results],
    rotation=45,
)
plt.xlabel("Model")
plt.ylabel("Validation AUC-ROC")
plt.title("Validation AUC-ROC for Different Models")
plt.tight_layout()
plt.savefig(os.path.join(visualizations_dir, "validation_auc_comparison.png"))
plt.close()

Found 2003 images belonging to 7 classes.
63/63 [==============================] - 9s 125ms/step
Confusion Matrix for InceptionV3_Adam:
[[  9   3   5  35   4   6   3]
 [ 10   4   7  65   3   7   7]
 [ 16   6  12 144  11  16  15]
 [  4   0   0  16   1   1   1]
 [ 28   4  12 140  17  11  11]
 [151  35  85 838  74  97  61]
 [  4   2   0  16   2   3   1]]
Classification Report for InceptionV3_Adam:
              precision    recall  f1-score   support

       akiec       0.04      0.14      0.06        65
         bcc       0.07      0.04      0.05       103
         bkl       0.10      0.05      0.07       220
          df       0.01      0.70      0.03        23
         mel       0.15      0.08      0.10       223
          nv       0.69      0.07      0.13      1341
        vasc       0.01      0.04      0.02        28

    accuracy                           0.08      2003
   macro avg       0.15      0.16      0.07      2003
weighted avg       0.49      0.08      0.11      2003

10/10

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i